In [1]:
import pickle
import matplotlib.pyplot as plt

In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import optimizers
import numpy as np
from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


In [3]:
mnist = keras.datasets.mnist
(mnist_train_x, mnist_train_y), (mnist_test_x, mnist_test_y)\
    = mnist.load_data()
def MNIST_To_CIFAR_FORM(mnist_train_x, mnist_train_y,mnist_test_x, mnist_test_y):
    """
    Change the one-channel to RBG-channel on mnist_train_x and mnist_test_x
    Change the shape of mnist_train_y and mnist_test_y from (length) to (length,1)
    ---------------------------------------
    inputs:
    mnist_train_x, mnist_train_y,mnist_test_x, mnist_test_y which is all multi-dimension array
    It is recommended to use the following way to import the data
    ========================== codes ==========================
    mnist = keras.datasets.mnist
    (mnist_train_x, mnist_train_y), (mnist_test_x, mnist_test_y)\
    = mnist.load_data()
    ========================== codes ==========================
    outputs:
    mnist_train_RGB_x, M_train_y, mnist_test_RGB_x, M_test_y 
    """
    from skimage import exposure
    import imutils
    B= []
    for i in range(len(mnist_train_x)):
        A = mnist_train_x[i]
        A = exposure.rescale_intensity(A, out_range=(0, 255))
        A = imutils.resize(A, width=32)
        B.append(A)
    B = np.array(B)

    mnist_train_RGB_x = np.repeat(B[:,:, :, np.newaxis], 3, axis=3)
    B= []
    for i in range(len(mnist_test_x)):
        A = mnist_test_x[i]
        A = exposure.rescale_intensity(A, out_range=(0, 255))
        A = imutils.resize(A, width=32)
        B.append(A)
    B = np.array(B)

    mnist_test_RGB_x = np.repeat(B[:,:, :, np.newaxis], 3, axis=3)
    M_train_y = np.array([[mnist_train_y[i]] for i in range(len(mnist_train_y))])
    M_test_y = np.array([[mnist_test_y[i]] for i in range(len(mnist_test_y))])
    return mnist_train_RGB_x, M_train_y, mnist_test_RGB_x, M_test_y
mnist_train_RGB_x, M_train_y, mnist_test_RGB_x, M_test_y = MNIST_To_CIFAR_FORM(mnist_train_x, mnist_train_y,mnist_test_x, mnist_test_y)

In [5]:
M_train_y.shape

(60000, 1)

In [6]:
def separater_MNIST(MNIST, num):
    mnist_train_x, M_train_y, mnist_test_x, M_test_y = MNIST
    M_train_y = M_train_y.reshape(-1)
    M_test_y = M_test_y.reshape(-1)
    MNIST_OD = [mnist_train_x[M_train_y == num], M_train_y[M_train_y == num].reshape((-1,1)), mnist_test_x[M_test_y == num], M_test_y[M_test_y == num].reshape((-1,1))]
    MNIST_ID = [mnist_train_x[M_train_y != num], M_train_y[M_train_y != num].reshape((-1,1)), mnist_test_x[M_test_y != num], M_test_y[M_test_y != num].reshape((-1,1))]
    return MNIST_ID, MNIST_OD

In [7]:
MNIST_ID, MNIST_OD = separater_MNIST([mnist_train_RGB_x, M_train_y, mnist_test_RGB_x, M_test_y], 9)

In [20]:
class MNIST_0_8_vgg:
    def __init__(self,train=True):
        self.num_classes = 9
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('MNIST_vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 25
        learning_rate = 0.001
        lr_decay = 1e-5
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        x_train,y_train,x_test,y_test = MNIST_ID
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train, x_test = self.normalize(x_train, x_test)

        y_train = keras.utils.to_categorical(y_train, self.num_classes)
        y_test = keras.utils.to_categorical(y_test, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        sgd = optimizers.adam(lr=learning_rate)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test, y_test),callbacks=[reduce_lr],verbose=2)
        model.save_weights('MNIST_vgg.h5')
        return model

if __name__ == '__main__':


    x_train, y_train, x_test, y_test = MNIST_ID
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    y_train = keras.utils.to_categorical(y_train, 9)
    y_test = keras.utils.to_categorical(y_test, 9)

    model = MNIST_0_8_vgg()

    predicted_x = model.predict(x_test)
    residuals = np.argmax(predicted_x,1)!=np.argmax(y_test,1)

    loss = sum(residuals)/len(residuals)
    print("the validation 0/1 loss is: ",loss)

Epoch 1/25
 - 49s - loss: 3.5600 - acc: 0.5720 - val_loss: 3.0456 - val_acc: 0.7247
Epoch 2/25
 - 43s - loss: 1.9958 - acc: 0.9195 - val_loss: 1.5546 - val_acc: 0.9687
Epoch 3/25
 - 43s - loss: 1.3823 - acc: 0.9514 - val_loss: 1.1216 - val_acc: 0.9641
Epoch 4/25
 - 44s - loss: 0.9979 - acc: 0.9587 - val_loss: 0.8148 - val_acc: 0.9761
Epoch 5/25
 - 43s - loss: 0.7740 - acc: 0.9624 - val_loss: 0.6601 - val_acc: 0.9721
Epoch 6/25
 - 43s - loss: 0.6533 - acc: 0.9624 - val_loss: 0.6194 - val_acc: 0.9681
Epoch 7/25
 - 43s - loss: 0.5686 - acc: 0.9648 - val_loss: 0.4820 - val_acc: 0.9802
Epoch 8/25
 - 44s - loss: 0.5619 - acc: 0.9615 - val_loss: 0.4898 - val_acc: 0.9781
Epoch 9/25
 - 49s - loss: 0.5523 - acc: 0.9625 - val_loss: 0.5586 - val_acc: 0.9582
Epoch 10/25
 - 50s - loss: 0.5468 - acc: 0.9627 - val_loss: 0.5389 - val_acc: 0.9734
Epoch 11/25
 - 48s - loss: 0.5717 - acc: 0.9613 - val_loss: 0.5378 - val_acc: 0.9667
Epoch 12/25
 - 44s - loss: 0.5962 - acc: 0.9605 - val_loss: 0.5047 - val_a

In [23]:
import pickle
pickle.dump(model, open("MNIST0-8-VGG1.pkl", "wb"))

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
def get_the_orignal_index_after_ranking(arr, isReverse = True):
    """
    This function return the orignal index of the after the array is sorted
    inputs:
    arr || one dimension list or ndarray
    isReverse || boolean, if it is "True" the rank is decending; if it is "False" the rank is ascending 
    outputs:
    A || an arr including the orignal index before ranking
    ========================= examples =========================
    For example, arr = [4,7,2,9]
    we have the mapping relationship:
    index    value
      0        4
      1        7
      2        2
      3        9
    After sorting, say decendingly, we have:
    orignal_index     value
      3                 9
      1                 7
      0                 4
      2                 2    
    the result is for this function is [3,1,0,2].
    """
    import operator
    similarity_dict = dict(zip(list(range(len(arr))),arr))
    sorted_similarity_dict = sorted(similarity_dict.items(), reverse=isReverse, key=operator.itemgetter(1))
    A = [sorted_similarity_dict[i][0] for i in range(len(arr))]
    return A

In [22]:
def find_idx(arr, target):
    ans = []
    for i in range(len(arr)):
        if arr[i] == target:
            ans.append(i)
    return ans
def get_submax(arr):
    arr = np.array(arr)
    MAX = np.max(arr)
    idx = find_idx(arr, MAX)
    arr_without_max = np.delete(arr,idx)
    return np.max(arr_without_max)
def find_statistics(Prob_Mat):
    Prob_diff = []
    MAX_Prob_Mat = []
    MAX_Prob_Mat_idx = []
    subMAX_Prob_Mat = []
    subMAX_Prob_Mat_idx = []
    for i in range(len(Prob_Mat)):
        MAX = np.max(Prob_Mat[i])
        MAX_idx = find_idx(Prob_Mat[i], MAX)[0]
        subMAX = get_submax(Prob_Mat[i])
        subMAX_idx = find_idx(Prob_Mat[i], subMAX)[0]
        prob_difference = MAX - subMAX
        Prob_diff.append(prob_difference)
        MAX_Prob_Mat.append(MAX)
        subMAX_Prob_Mat.append(subMAX)
        MAX_Prob_Mat_idx.append(MAX_idx)
        subMAX_Prob_Mat_idx.append(subMAX_idx)
    return Prob_diff,MAX_Prob_Mat,MAX_Prob_Mat_idx,subMAX_Prob_Mat,subMAX_Prob_Mat_idx
def separate_one_class(target_class_label, x_train, y_train, x_test, y_test):
    with_train_idx = find_idx(y_train, target_class_label)
    with_test_idx = find_idx(y_test, target_class_label)
    without_train_idx = list(set(range(len(y_train))).difference(set(with_train_idx)))
    without_test_idx = list(set(range(len(y_test))).difference(set(with_test_idx)))
    with_train = x_train[with_train_idx]
    with_test = x_test[with_test_idx]
    without_train = x_train[without_train_idx]
    without_test = x_test[without_test_idx]
    with_train_y = y_train[with_train_idx]
    with_test_y = y_test[with_test_idx]
    without_train_y = y_train[without_train_idx]
    without_test_y = y_test[without_test_idx]
    return with_train, with_train_y, without_train, without_train_y, with_test, with_test_y, without_test, without_test_y
def minkowski_distance(x,y,n):
    if np.isinf(n):
        if n>0:
            return np.max(np.abs(x-y))
        else:
            return np.min(np.abs(x-y))
    else:
        return np.power(np.sum(np.power(np.abs(x-y),n)),1/n)
def minkowski_similarity(x,Y,n):
    arr = []
    for y in Y:
        arr.append(minkowski_distance(x,y,n))
    return np.array(arr)

def get_KNN_stats(k,testarr_one_sample, testarr_waiting_to_compare, 
                  testarr_waiting_to_compare_label, Model, similarity_method = 'cosine_similarity', minkowski_power = 2):
    """
    Inputs Example:
    k = 50
    testarr_one_sample = [mnist_train_RGB_x[0]]
    testarr_waiting_to_compare = [C_x_train[i] for i in range(5000)]
    testarr_waiting_to_compare_label = C_y_train[:5000].reshape(5000)
    Model = C_VGG_Model1.model
    
    Inputs:
    k: int, the number of the nearest neighbour
    testarr_one_sample: multi-dimensional ndarray, shape = (1,num_pixel_x,num_pixel_y,num_channel)
    testarr_waiting_to_compare_label: multi-dimensional ndarray, shape = (num_neighbour_candidate,num_pixel_x,num_pixel_y,num_channel)
    testarr_waiting_to_compare_label: one-dimensional ndarray, shape = (num_neighbour_candidate,)
    Model: keras backend model
    similarity_method: String, 'cosine_similarity', 'minkowski_similarity'. Default = 'cosine_similarity'
    minkowski_power: int, the p-value in the minkowski_distance. Only useful when similarity_method = 'minkowski_similarity'
    
    Outputs:
    similarity: one-dimensional ndarray, shape = (num_neighbour_candidate,)
    K_nearest_neighbour_orignal_label: one-dimensional ndarray, shape = (k,)
    K_nearest_neighbour: multi-dimensional ndarray, shape = (k,num_pixel_x,num_pixel_y,num_channel)
    KNN_oringal_class: dictionary, counts of the orignal class
    max_ratio_KNN_from_one_class: float, the max of the ratio of KNN are from one class
    """
    from keras import backend as K
    testarr_waiting_to_compare = np.array(testarr_waiting_to_compare)
    inp = Model.model.input                                           # input placeholder
    outputs = Model.model.layers[55].output          # all layer outputs
    functors = K.function([inp, K.learning_phase()], [outputs])   # evaluation functions

    # Testing
    test1 = testarr_one_sample
    layer_outs_one_sample = functors([test1, 0.])
    layer_outs_one_sample = np.array(layer_outs_one_sample)[0]

    test2 = testarr_waiting_to_compare
    layer_outs_waiting_to_compare = functors([test2, 0.])
    layer_outs_waiting_to_compare = np.array(layer_outs_waiting_to_compare)[0]
    if similarity_method == 'cosine_similarity':
        from sklearn.metrics.pairwise import cosine_similarity
        similarity = cosine_similarity(layer_outs_one_sample, layer_outs_waiting_to_compare)
        similarity = np.array(similarity[0])
        K_nearest_neighbour_orignal_label = get_the_orignal_index_after_ranking(similarity)[:k]
    elif similarity_method == 'minkowski_similarity':
        similarity = minkowski_similarity(layer_outs_one_sample, layer_outs_waiting_to_compare, minkowski_power)
        K_nearest_neighbour_orignal_label = get_the_orignal_index_after_ranking(similarity, isReverse = False)[:k]
    else:
        raise Exception("invalid similarity method")
    K_nearest_neighbour_orignal_label = np.array(K_nearest_neighbour_orignal_label)
    K_nearest_neighbour = testarr_waiting_to_compare[K_nearest_neighbour_orignal_label]
    from collections import Counter
    KNN_oringal_class = Counter(testarr_waiting_to_compare_label[K_nearest_neighbour_orignal_label])
    max_ratio_KNN_from_one_class = max(KNN_oringal_class.values())/k
    import operator
    max_KNN_class_label = max(KNN_oringal_class.items(), key=operator.itemgetter(1))[0]
    return similarity, K_nearest_neighbour_orignal_label, K_nearest_neighbour, KNN_oringal_class, max_ratio_KNN_from_one_class, max_KNN_class_label

In [14]:
def show(columns,rows,arr):
    w=10
    h=10
    fig=plt.figure(figsize=(8, 8))
    for i in range(1, columns*rows +1):
        img = arr[i-1]
        fig.add_subplot(rows, columns, i)
        plt.imshow(img)
    plt.show()
    return

In [15]:
def compare_representations(testarr_one_sample, testarr_waiting_to_compare,testarr_waiting_to_compare_label, MODEL, similarity_method = 'minkowski_similarity', minkowski_power = 2):
    arr_KNN_from_same_class_ratio = []
    arr_KNN_max_class_label = []
    arr_KNN_similarity = []
    testarr_waiting_to_compare = testarr_waiting_to_compare[:5000]
    testarr_waiting_to_compare_label = testarr_waiting_to_compare_label[:5000].reshape(5000)
    for j in range(500):
        print("current sample: ", j)
        k = 50
        testarr_one_sample_1 = [testarr_one_sample[j]]
        result_l2 = get_KNN_stats(k,testarr_one_sample_1, testarr_waiting_to_compare, 
                  testarr_waiting_to_compare_label, MODEL, similarity_method = 'minkowski_similarity', minkowski_power = 2)
        arr_KNN_from_same_class_ratio.append(result_l2[4])
        arr_KNN_max_class_label.append(result_l2[5])
        arr_KNN_similarity.append(sorted(result_l2[0],reverse = False))
    pred_labels = MODEL.predict(testarr_one_sample[:500])
    import pandas as pd
    df = pd.DataFrame({'arr_KNN_max_class_label':arr_KNN_max_class_label,
                   'arr_KNN_from_same_class_ratio':arr_KNN_from_same_class_ratio,
                   'predicted_label':find_statistics(pred_labels)[2],
                  'predicted_prob':find_statistics(pred_labels)[1],})
    return arr_KNN_similarity, arr_KNN_from_same_class_ratio, arr_KNN_max_class_label, df

In [59]:
def get_55representation(Model, inputs):
    inp = Model.model.input                                           # input placeholder
    outputs = Model.model.layers[55].output          # all layer outputs
    functors = K.function([inp, K.learning_phase()], [outputs])   # evaluation functions
    
    output = functors([inputs, 0.])
    output = np.array(output)[0]
    return output

def split_apart_5000_get_re(dataset, dataset_name, model):
    n = len(dataset)/5000
    result = []
    for i in range(int(n)):
        print("Dealing with "+dataset_name+" batch "+str(i+1))
        split = dataset[i*5000:(1+i)*5000]
        split_representation = get_55representation(model,split)
        for x in split_representation:
            result.append(x)
    result = np.array(result)
    #print(result.shape)
    return result
def split_into_mod5000(dataset):
    n = len(dataset)/5000
    return dataset[:int(n)*5000], dataset[int(n)*5000:]
def split_apart_get_re(dataset, model):
    print("Dealing with the residule.")
    return get_55representation(model,dataset)

def split_main(dataset, dataset_name, model):
    if len(dataset)>5000:
        A, B = split_into_mod5000(dataset)
        return np.append(split_apart_5000_get_re(A, dataset_name, model), split_apart_get_re(B, model),axis = 0)
    else:
        return split_apart_get_re(dataset, model)

In [60]:
M_0_8_representation = {}
M_0_8_representation['MNIST0_8_train'] = []
M_0_8_representation['MNIST0_8_test'] = []
M_0_8_representation['MNIST_9_train'] = []
M_0_8_representation['MNIST_9_test'] = []

datasets = [MNIST_ID[0], MNIST_ID[2], MNIST_OD[0], MNIST_OD[2]]
datasets_name = ['MNIST0_8_train', 'MNIST0_8_test', 'MNIST_9_train', 'MNIST_9_test']
#var_name_trans = {'mnist_train_RGB_x':'MNIST_train','mnist_test_RGB_x':'MNIST_test','C_x_train':'CIFAR_train','C_x_test':'CIFAR_test'}



for i in range(len(datasets)):
    M_0_8_representation[datasets_name[i]] = split_main(datasets[i],datasets_name[i],model)
    #C_representation[var_name_trans[datasets_name[i]]] = split_apart_5000_get_re(datasets[i],datasets_name[i],C_VGG_Model3)

Dealing with MNIST0_8_train batch 1
Dealing with MNIST0_8_train batch 2
Dealing with MNIST0_8_train batch 3
Dealing with MNIST0_8_train batch 4
Dealing with MNIST0_8_train batch 5
Dealing with MNIST0_8_train batch 6
Dealing with MNIST0_8_train batch 7
Dealing with MNIST0_8_train batch 8
Dealing with MNIST0_8_train batch 9
Dealing with MNIST0_8_train batch 10
Dealing with the residule.
Dealing with MNIST0_8_test batch 1
Dealing with the residule.
Dealing with MNIST_9_train batch 1
Dealing with the residule.
Dealing with the residule.


In [39]:
def shape(List):
    return np.array(List).shape

In [65]:
import pickle
pickle.dump(M_0_8_representation, open("M_0_8_representation.pkl", "wb"))